# Exemplo 14: Telecom - Qualidade de Rede (Dask)

Este notebook demonstra a análise de logs de chamadas (CDR) para identificar torres com problemas de qualidade usando **Dask**.

**Cenário**: Calcular a taxa de queda de chamadas (Drop Call Rate) por torre de celular.

## 1. Configuração

In [ ]:
# Instalar Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar e Instalar Spark
!wget https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz && tar xf spark-3.5.0-bin-hadoop3.tgz

# Baixar e Instalar Kafka
!wget https://archive.apache.org/dist/kafka/3.6.1/kafka_2.13-3.6.1.tgz && tar xf kafka_2.13-3.6.1.tgz

# Instalar pacotes Python
!pip install -q findspark pyspark kafka-python

In [ ]:
!pip install -q dask[complete] distributed

In [ ]:
from dask.distributed import Client
client = Client()
client

## 2. Gerar CDRs (Call Detail Records)

In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os

os.makedirs('telecom_data', exist_ok=True)

def generate_cdrs(file_id):
    num_records = 10000
    towers = [f'TOWER-{i:03d}' for i in range(100)]
    
    df = pd.DataFrame({
        'call_id': range(num_records),
        'tower_id': np.random.choice(towers, num_records),
        'duration_sec': np.random.exponential(120, num_records),
        'status': np.random.choice(['SUCCESS', 'DROP', 'FAIL'], num_records, p=[0.9, 0.05, 0.05])
    })
    
    # Simular problema na TOWER-099
    mask = df['tower_id'] == 'TOWER-099'
    df.loc[mask, 'status'] = np.random.choice(['SUCCESS', 'DROP'], mask.sum(), p=[0.6, 0.4])
    
    df.to_csv(f'telecom_data/cdr_{file_id}.csv', index=False)

print("Gerando CDRs...")
for i in range(10):
    generate_cdrs(i)

## 3. Análise de Qualidade

In [ ]:
ddf = dd.read_csv('telecom_data/cdr_*.csv')

# Calcular total de chamadas e quedas por torre
stats = ddf.groupby('tower_id').agg(
    total_calls=('call_id', 'count'),
    dropped_calls=('status', lambda x: (x == 'DROP').sum())
)

# Calcular taxa de queda
stats_computed = stats.compute()
stats_computed['drop_rate'] = stats_computed['dropped_calls'] / stats_computed['total_calls']

# Identificar torres problemáticas (> 10% de queda)
bad_towers = stats_computed[stats_computed['drop_rate'] > 0.10]

print("Torres com alta taxa de queda:")
print(bad_towers.sort_values('drop_rate', ascending=False))

In [ ]:
client.close()